In [ ]:
import pandas as pd

In [ ]:
gb = pd.read_csv("../input/garbage/spotgarbage-GINI-master/spotgarbage/garbage-queried-images.csv")
no_gb = pd.read_csv("../input/garbage/spotgarbage-GINI-master/spotgarbage/non-garbage-queried-images.csv")

In [ ]:
gb = gb.drop(['startX','startY','endX','endY','query'],axis=1)
no_gb = no_gb.drop(['query'],axis=1)

In [ ]:
gb.head()

In [ ]:
no_gb.head()

In [ ]:
no_gb["label"] = 0.0

In [ ]:
gb["label"].isna().sum()

In [ ]:
PATH = "../input/garbage/spotgarbage-GINI-master/spotgarbage"

In [ ]:
import glob
import cv2
import numpy as np
import tensorflow as tf
import os
import pandas as pd

In [ ]:
print(gb.shape)
print(no_gb.shape)

In [ ]:
cols = ["path","non-garbage-queried-images" ,"garbage-queried-images"]

In [ ]:
data = pd.DataFrame(columns = cols, index= range(0,2512))

In [ ]:
data[cols] = 0
data.head()

In [ ]:
for x in os.walk(PATH):
    directory = x[1]
    break
print(directory)

In [ ]:
c = 0
for dir in directory:
    print(PATH +dir)
    for i in glob.glob(PATH+ "/" +dir +'/*', recursive=True):
        data.loc[c,"path"] = i
        data.loc[c,dir] =1
        c = c + 1


print(str(c))

In [ ]:
data.head()

In [ ]:
data[data["ambiguous-annotated-images"] == 1.0].shape

In [ ]:
df = data[data["ambiguous-annotated-images"] != 1.0]

In [ ]:
df.shape

In [ ]:
df = df.drop(["ambiguous-annotated-images"], axis=1)

In [ ]:
df.head()

In [ ]:
from fastai import *
from fastai.vision import *
from torch import  *
from fastai.callbacks import *
import warnings
import torch
import torchvision.models as torch_models
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

In [ ]:
!mkdir "garbage-queried-images"
!mkdir "non-garbage-queried-images"

In [ ]:
PATH = "/kaggle/working"

In [ ]:
!cp "../input/garbage/spotgarbage-GINI-master/spotgarbage/garbage-queried-images/"*/* "/kaggle/working/garbage-queried-images/"

In [ ]:
!cp "../input/garbage/spotgarbage-GINI-master/spotgarbage/non-garbage-queried-images/"*/* "/kaggle/working/non-garbage-queried-images/"

In [ ]:
!ls "/kaggle/working/garbage-queried-images/" | wc -l

In [ ]:
!ls "/kaggle/working/non-garbage-queried-images/" | wc -l

In [ ]:
np.random.seed(42)
data=ImageDataBunch.from_folder(PATH,train="train",
                                ds_tfms=get_transforms(do_flip=True,flip_vert=True),valid_pct=0.1,size=350,num_workers=0,bs=32).normalize(imagenet_stats)

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
classes=["non-garbage-queried-images","garbage-queried-images"]
for c in classes :
    print(c)
    verify_images (PATH + c, max_workers=2)

In [ ]:
learn= cnn_learner(data,torch_models.wide_resnet50_2, metrics=[accuracy,Recall(),error_rate])

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3, max_lr=slice(1e-4,1e-3),callbacks=[SaveModelCallback(learn, every='improvement', monitor='accuracy')])

In [ ]:
interpreter = ClassificationInterpretation.from_learner(learn)
interpreter.most_confused(min_val=11)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=slice(1e-6),callbacks=[SaveModelCallback(learn, every='improvement', monitor='accuracy')])

In [ ]:
learn.save('model')

In [ ]:
learn.export()